# 앙상블 (mmdetection model + ultralytics YOLOv8) 가이드

## 필요 라이브러리 한번에 다운로드

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
!pip install -e ../mmdetection

In [ ]:
!mim install "mmengine>=0.7.0"
!mim install "mmcv>=2.0.0rc4"

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../mmdetection')

## 데이터 준비

In [ ]:
import cv2
import matplotlib.pyplot as plt

# 이미지 파일 경로
image_path = "./data/user_1/driver_seat_mat.jpg"

# 이미지 불러오기
img = cv2.imread(image_path)

final_width = img.shape[1]
final_height = img.shape[0]

# 이미지를 새로운 크기로 조정
mmdetection_img = cv2.resize(img, (1333, 800))
yolov8_img = cv2.resize(img, (640, 640))
final_img = cv2.resize(img, (final_width, final_height))

# Inference

In [ ]:
from inference.predict import *

## mmdetection

In [ ]:
config_file = '../mmdetection/configs/dino/dino-4scale_r50_8xb2-60e_coco_all_v2.py'
mmdetection_checkpoint_file = './model/weights/mmdetection_dino_weights.pth'

In [ ]:
mmdetection_prediction = mmdetection_predict(mmdetection_img, config_file, mmdetection_checkpoint_file)

In [ ]:
mmdetection_prediction

## YOLOv8 모델

In [ ]:
yolov8_checkpoint_file = "./model/weights/yolov8_weights.pt"

In [ ]:
yolov8_prediction = yolov8_predict(yolov8_img, yolov8_checkpoint_file)

In [ ]:
yolov8_prediction

## 결과 시각화

In [ ]:
class_labels = ['bag', 'bottle', 'cafe-cup', 'can', 'dirty', 'keys', 'seat-dirty', 'socar-tissue', 'stain', 'trash', 'valuable']

# 각 클래스에 대한 색상 지정
class_colors = {
    'bag': (255, 0, 0),       # 빨간색
    'bottle': (0, 255, 0),    # 초록색
    'cafe-cup': (0, 0, 255),  # 파란색
    'can': (255, 255, 0),     # 노란색
    'dirty': (255, 0, 255),   # 자홍색
    'keys': (0, 255, 255),    # 청록색
    'seat-dirty': (255, 165, 0),      # 오렌지색
    'socar-tissue': (128, 0, 128),    # 보라색
    'stain': (128, 128, 0),    # 올리브색
    'trash': (0, 128, 128),    # 틸색
    'valuable': (255, 192, 203)  # 분홍색
}

def draw_detection_results(image, detection_results, img_width, img_height, normalize=False):
    # 복제하여 원본 이미지 변경 방지
    image_with_boxes = image.copy()

    for label, score, bbox in zip(detection_results['labels'], detection_results['scores'], detection_results['bboxes']):        
        if normalize:
            x1_norm, y1_norm, x2_norm, y2_norm = bbox

            # 정규화된 좌표를 이미지 크기에 맞게 변환
            x1 = int(x1_norm * img_width)
            y1 = int(y1_norm * img_height)
            x2 = int(x2_norm * img_width)
            y2 = int(y2_norm * img_height)
        else:
            x1, y1, x2, y2 = map(int, bbox)
        
        # 클래스 이름 가져오기
        class_name = class_labels[label]

        # 클래스에 대응하는 색상 가져오기
        color = class_colors.get(class_name, (0, 0, 0))  # 기본은 검은색

        # 상자 그리기
        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), color, 2)

        # 텍스트 추가 (클래스명과 신뢰도)
        label_text = f"Class: {class_name}, Score: {score:.2f}"
        cv2.putText(image_with_boxes, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image_with_boxes

## dino 모델 추론 결과 그리기

In [ ]:
# 결과를 이미지에 그리기
mmdetection_image_with_boxes = draw_detection_results(mmdetection_img, mmdetection_prediction, 1333, 800)

plt.figure(figsize=(18, 15))
plt.imshow(cv2.cvtColor(mmdetection_image_with_boxes, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축 표시하지 않음
plt.show()

## yolov8 모델 추론 결과 그리기

In [ ]:
# 결과를 이미지에 그리기
yolov8_image_with_boxes = draw_detection_results(yolov8_img, yolov8_prediction, 640, 640)

plt.figure(figsize=(18, 15))
plt.imshow(cv2.cvtColor(yolov8_image_with_boxes, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축 표시하지 않음
plt.show()

## wbf 앙상블

In [ ]:
wbf_prediction = ensemble_predict(mmdetection_prediction, yolov8_prediction)

In [ ]:
wbf_prediction

In [ ]:
def denormalize_ensemble_prediction(detection_result, img_width, img_height):
    denormalized_wbf_prediction = {'labels': [], 'scores': [], 'bboxes': []}

    for label, score, bbox in zip(detection_result['labels'], detection_result['scores'], detection_result['bboxes']):
        normalized_xmin, normalized_ymin, normalized_xmax, normalized_ymax = bbox

        # 원래 이미지 크기에 따른 비율 계산
        x_scale = img_width
        y_scale = img_height

        # 좌표를 원래 크기로 되돌리기
        denormalized_xmin = normalized_xmin * x_scale
        denormalized_ymin = normalized_ymin * y_scale
        denormalized_xmax = normalized_xmax * x_scale
        denormalized_ymax = normalized_ymax * y_scale

        denormalized_wbf_prediction['labels'].append(label)
        denormalized_wbf_prediction['scores'].append(score)
        denormalized_wbf_prediction['bboxes'].append([denormalized_xmin, denormalized_ymin, denormalized_xmax, denormalized_ymax])

    return denormalized_wbf_prediction

# 함수 호출 및 결과 출력
expanded_prediction = denormalize_ensemble_prediction(wbf_prediction, final_width, final_height)
print(expanded_prediction)

In [ ]:
# 이미지에 확장된 박스 그리기
expanded_image_with_boxes = draw_detection_results(final_img, expanded_prediction, final_width, final_height)

plt.figure(figsize=(18, 15))
plt.imshow(cv2.cvtColor(expanded_image_with_boxes, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축 표시하지 않음
plt.show()

## 최종 앙상블 모델 추론 결과 그리기

In [ ]:
# 결과를 이미지에 그리기
final_image_with_boxes = draw_detection_results(final_img, wbf_prediction, final_width, final_height, True)

plt.figure(figsize=(18, 15))
plt.imshow(cv2.cvtColor(final_image_with_boxes, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축 표시하지 않음
plt.show()

# 이미지 결과 비교

In [ ]:
mmdetection_image_with_boxes = draw_detection_results(mmdetection_img, mmdetection_prediction, 1333, 800)
yolov8_image_with_boxes = draw_detection_results(yolov8_img, yolov8_prediction, 640, 640)
final_image_with_boxes = draw_detection_results(final_img, wbf_prediction, final_width, final_height, True)

target_size = (800, 1200)

mmdetection_image_with_boxes_resized = cv2.resize(mmdetection_image_with_boxes, target_size)
yolov8_image_with_boxes_resized = cv2.resize(yolov8_image_with_boxes, target_size)
final_image_with_boxes_resized = cv2.resize(final_image_with_boxes, target_size)

fig, axes = plt.subplots(1, 3, figsize=(18, 18))

axes[0].imshow(cv2.cvtColor(mmdetection_image_with_boxes_resized, cv2.COLOR_BGR2RGB))
axes[0].set_title('dino Results')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(yolov8_image_with_boxes_resized, cv2.COLOR_BGR2RGB))
axes[1].set_title('Yolov8 Results')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(final_image_with_boxes_resized, cv2.COLOR_BGR2RGB))
axes[2].set_title('ensemble Results')
axes[2].axis('off')

plt.tight_layout()
plt.show()